In [215]:
import numpy as np
import cv2
from PIL import Image
import imutils
from pytesseract import image_to_string
import csv

def order_points(pts):
    rect = np.zeros((4, 2), dtype = "float32")
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def four_point_transform(image, pts):
    rect = order_points(pts)
    tl, tr, br, bl = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype = "float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

def get_photo(card, ID):
    card_height, card_width, channels = card.shape
    photo_h1 = round(0.28*card_height)
    photo_h2 = round(0.76*card_height)
    photo_w1 = round(0.0036*card_width)
    photo_w2 = round(0.233*card_width)
    photo = card[photo_h1: photo_h2, photo_w1: photo_w2]
    cv2.imwrite('Result/Photo.jpg', photo)
    cv2.imwrite('Data/{}.jpg'.format(ID), photo)
    
def get_id(card):
    card_height, card_width, channels = card.shape
    ID_h1 = round(0.9*card_height)
    ID_h2 = round(0.9799*card_height)
    ID_w1 = round(0.003*card_width)
    ID_w2 = round(0.26*card_width)
    ID = card[ID_h1: ID_h2, ID_w1: ID_w2]
    ID = cv2.cvtColor(ID, cv2.COLOR_BGR2GRAY)
    retval2, ID = cv2.threshold(ID, 125, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    cv2.imwrite('Result/ID.jpg', ID)
    return image_to_string(Image.open('Result/ID.jpg'), lang='vie')
    
def get_name(card):
    card_height, card_width, channels = card.shape
    name_h1 = round(0.356*card_height)
    name_h2 = round(0.47*card_height)
    name_w1 = round(0.238*card_width)
    name_w2 = round(0.96*card_width)
    name = card[name_h1: name_h2, name_w1: name_w2]
    name = cv2.cvtColor(name, cv2.COLOR_BGR2GRAY)
    retval2, name = cv2.threshold(name, 125, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    cv2.imwrite('Result/Name.jpg', name)
    return image_to_string(Image.open('Result/Name.jpg'), lang='vie')

def get_birth_day(card):   
    card_height, card_width, channels = card.shape
    dob_h1 = round(0.45*card_height)
    dob_h2 = round(0.545*card_height)
    dob_w1 = round(0.238*card_width)
    dob_w2 = round(0.647*card_width)
    dob = card[dob_h1: dob_h2, dob_w1: dob_w2]
    dob = cv2.cvtColor(dob, cv2.COLOR_BGR2GRAY)
    retval2, dob = cv2.threshold(dob, 125, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    cv2.imwrite('Result/Date_of_Birth.jpg', dob)
    return image_to_string(Image.open('Result/Date_of_Birth.jpg'), lang='vie')

def get_info():
    name_img = input()
    img_path = 'test/' + name_img
    img = cv2.imread(img_path)
    h, w, c = img.shape
    while(h > 3000 or w > 3000):
        h = int(h/2.5)
        w = int(w/2.5)
        c = 1
    if c == 1:
        img = cv2.resize(img, (w, h))
    origin = img.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(gray, 100, 250)
    #ret, edges = cv2.threshold(edges, 0, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    cnts = cv2.findContours(closed.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if imutils.is_cv2() else cnts[1]
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.01*peri, True)
        screenCnt = 0
        if len(approx == 4):
            screenCnt = approx
            break
    cv2.drawContours(img, [screenCnt], -1, (0, 255, 0), 2)
    card = four_point_transform(origin, screenCnt.reshape(4, 2))
    card_height, card_width, channels = card.shape
    cv2.imwrite('Result/Card.jpg', card)
    cv2.imwrite('Result/Image_detected.jpg', img)
    card = cv2.imread('Result/Card.jpg')
    ID = get_id(card)[get_id(card).find('B') : 11]
    get_photo(card, ID)
    Name = get_name(card)[get_name(card).rfind(':') + 2 :]
    Date_of_Birth = get_birth_day(card)[get_birth_day(card).rfind(':') + 2 :]
    result = [Name, ID, Date_of_Birth]
    return result

def add_info():
    info = get_info()
    with open('Data/test.csv', 'a', newline='') as csvfile:
        data = csv.writer(csvfile)
        data.writerow([info[0], info[1], info[2]])
    print(info)
        
def check(ID):
    with open('Data/test.csv', 'r') as f:
        reader = csv.reader(f)
        data = list(reader)
    for i in range (len(data)):
        if ID == data[i][1]:
            print(data[i])
            return True
    add_info()
    

In [231]:
ID = 'B17DCC09'

In [232]:
def check_form(ID):
    check_list = ['DCCN', 'DCAT', 'DCVT', 'DCKT', 'DCTM', 'DCDT', 'DCQT', 'DCMR', 'DCTT', 'DCPT']
    if (ID[0] == 'B' or ID[0] == [E]) and ID[1:3].isdigit() and ID[3:7] in check_list and ID[7:11].isdigit():
        return True
    else:
        return False

In [233]:
check_form(ID)

False